# 📊 QEPC NBA Dashboard

Interactive dashboard for:
- 🗓️ Selecting an upcoming NBA game
- 📊 Using injury-adjusted team strengths
- λ Running QEPC simulations with calibrated lambdas
- 🎯 Viewing win probabilities, expected spread & total
- 🩺 Inspecting key injuries for the two teams


## 🧩 1. Environment & Core Imports


-----

In [ ]:
# qepc_dashboard.ipynb
# Cell 1: QEPC Dashboard setup

import os
import sys
from pathlib import Path

# Locate notebook_context so project_root is set
try:
    from notebook_context import *
    print("✅ Imported notebook_context directly.")
except ModuleNotFoundError:
    print("ℹ️ notebook_context not found on sys.path; trying to locate it...")

    cwd = Path.cwd()
    candidate_roots = [cwd, cwd.parent, cwd.parent.parent]

    found_root = None
    for root in candidate_roots:
        if (root / "notebook_context.py").exists():
            found_root = root
            break

    if found_root is None:
        raise ModuleNotFoundError(
            f"Could not find notebook_context.py in {cwd} or its parents. "
            f"Make sure this notebook lives under your qepc_project folder."
        )

    sys.path.insert(0, str(found_root))
    os.chdir(found_root)
    print(f"🔗 Added {found_root} to sys.path and changed working directory there.")

    from notebook_context import *
    print("✅ Imported notebook_context after path adjustment.")

# Fallback if notebook_context didn't define project_root
try:
    project_root
except NameError:
    project_root = Path.cwd()
    print("⚠️ 'project_root' was not defined by notebook_context; "
          "using current working directory as project_root instead.")

print("Project root:", project_root)

# Core imports
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML

import qepc_autoload as qa
from qepc.sports.nba.strengths_v2 import calculate_advanced_strengths
from qepc.sports.nba.sim import compute_lambda, run_qepc_simulation

# Global lambda scale (you can tweak this number manually if you want calibration)
GLOBAL_LAMBDA_SCALE = 1.0
print("GLOBAL_LAMBDA_SCALE =", GLOBAL_LAMBDA_SCALE)


------

## 📊 2. Load NBA Schedule

In [ ]:
# Cell 2: Load full NBA schedule

schedule = qa.load_nba_schedule()
print("Rows in full schedule:", len(schedule))

# Ensure gameDate is datetime
schedule["gameDate"] = pd.to_datetime(schedule["gameDate"], errors="coerce")

display(schedule.head())


-------------------------------

## 🧬 3. Team Strengths & Injury Overrides (Global)


In [ ]:
# Cell 3: Team Strengths & Injury Overrides (Global)

from datetime import datetime, timedelta

# 3.1 Base advanced strengths
advanced_team_strengths = calculate_advanced_strengths()
print(f"Loaded advanced strengths for {len(advanced_team_strengths)} teams.")

# 3.2 Load injury overrides (stack multiple sources)

base_dir = project_root / "data"

paths = [
    base_dir / "Injury_Overrides_live_official.csv",    # from nbainjuries (LATEST)
    base_dir / "Injury_Overrides_live_balldontlie.csv", # optional
    base_dir / "Injury_Overrides_live_espn.csv",        # optional
    base_dir / "Injury_Overrides_data_driven.csv",      # data-driven season-long
    base_dir / "Injury_Overrides.csv",                  # manual long-term
]

frames = []
for p in paths:
    if p.exists():
        print("Including injuries from:", p.name)
        df = pd.read_csv(p)
        df["__source"] = p.name
        frames.append(df)

if frames:
    injuries = pd.concat(frames, ignore_index=True)
    print("Total injury rows loaded:", len(injuries))

    # If a report timestamp exists, show the latest one
    if "Report_Timestamp" in injuries.columns:
        latest_ts = injuries["Report_Timestamp"].dropna().max()
        if pd.notna(latest_ts):
            print(f"🧬 Live injury snapshot timestamp: {latest_ts}")
else:
    injuries = None
    print("⚠️ No injury override files found.")

# 3.3 Apply injury impact to team strengths

team_strengths_for_lambda = advanced_team_strengths.copy()

if injuries is not None:
    if "Team" not in injuries.columns:
        raise ValueError("Injury overrides file(s) need a 'Team' column.")
    if "Impact" not in injuries.columns:
        injuries["Impact"] = 1.0

    def team_factor(series):
        prod = series.prod()
        return max(0.60, prod)  # don't crush below 60% of baseline

    team_factors = (
        injuries.groupby("Team")["Impact"]
        .apply(team_factor)
        .reset_index()
        .rename(columns={"Impact": "ORtg_factor"})
    )

    team_strengths_for_lambda = team_strengths_for_lambda.merge(
        team_factors, on="Team", how="left"
    )

    team_strengths_for_lambda["ORtg_factor"] = (
        team_strengths_for_lambda["ORtg_factor"].fillna(1.0)
    )

    team_strengths_for_lambda["ORtg_raw"] = team_strengths_for_lambda["ORtg"]
    team_strengths_for_lambda["ORtg"] = (
        team_strengths_for_lambda["ORtg_raw"] * team_strengths_for_lambda["ORtg_factor"]
    )

    print("Injury adjustments applied to team strengths.")
else:
    print("No injury adjustments applied (no overrides file).")

display(team_strengths_for_lambda.head())


----

## 📅 4. Upcoming Games & Game Selector


In [ ]:
# Cell 4: Upcoming games (next 3 days)

today = pd.Timestamp.today().normalize()
end_date = today + pd.Timedelta(days=3)

upcoming_games = schedule[
    (schedule["gameDate"] >= today) & (schedule["gameDate"] < end_date)
].copy()

upcoming_games = upcoming_games.sort_values("gameDate").reset_index(drop=True)

print(f"Upcoming games between {today.date()} and {end_date.date()}: {len(upcoming_games)}")

# Add a nice label for dropdowns
def make_label(row):
    dt = row["gameDate"]
    return f"{dt:%Y-%m-%d %I:%M %p} – {row['Away Team']} @ {row['Home Team']}"

upcoming_games["Label"] = upcoming_games.apply(make_label, axis=1)

display(
    upcoming_games[
        ["Label", "Date", "Time", "Away Team", "Home Team", "Venue", "Notes"]
    ]
)


----

## 📊 5. Team Strengths & Injuries – Upcoming Teams Only


In [ ]:
# Cell 5: Team Strengths & Injuries – Upcoming Teams Only

if upcoming_games.empty:
    print("No upcoming games in the next 3 days.")
else:
    upcoming_teams = sorted(
        set(upcoming_games["Home Team"]).union(set(upcoming_games["Away Team"]))
    )
    print("Teams with games in this window:", ", ".join(upcoming_teams))

    # Strengths snapshot
    strengths_subset = team_strengths_for_lambda[
        team_strengths_for_lambda["Team"].isin(upcoming_teams)
    ].copy()

    strength_cols_pref = ["Team", "ORtg", "DRtg", "Pace", "Volatility", "ORtg_factor"]
    strength_cols = [c for c in strength_cols_pref if c in strengths_subset.columns]

    print("\n📊 Team strengths for upcoming teams:")
    display(
        strengths_subset[strength_cols]
        .sort_values("Team")
        .reset_index(drop=True)
        .style.hide_index()
    )

    # Injury snapshot
    if injuries is not None:
        inj_subset = injuries[injuries["Team"].isin(upcoming_teams)].copy()

        if inj_subset.empty:
            print("\n🩺 Injury overrides: none for upcoming teams.")
        else:
            print("\n🩺 Injury overrides for upcoming teams:")
            inj_cols_pref = [
                "Team",
                "PlayerName",
                "Status",
                "Injury",
                "Impact",
                "EstReturn",
                "Source",
            ]
            inj_cols = [c for c in inj_cols_pref if c in inj_subset.columns]
            inj_subset = inj_subset[inj_cols].sort_values(
                ["Team", "PlayerName"]
            ).reset_index(drop=True)
            display(inj_subset.style.hide_index())
    else:
        print("\n🩺 Injury overrides: none loaded (no overrides file).")


-----

## 📊 6. QEPC Game Dashboard (Single Game View)


In [ ]:
# Cell 6: QEPC Game Dashboard (Single Game View)

if upcoming_games.empty:
    print("No upcoming games available for single-game view.")
else:
    output_area = widgets.Output()

    def run_qepc_for_game(row):
        """
        Given a single row from upcoming_games, build a 1-row games_to_model,
        compute lambdas (with injury-adjusted strengths), apply global calibration,
        run QEPC sim, and return the results.
        """
        games_to_model = row.to_frame().T.copy()

        # Compute lambda for this game
        lambda_df = compute_lambda(games_to_model, team_strengths_for_lambda)

        # Apply global lambda scale
        if GLOBAL_LAMBDA_SCALE != 1.0:
            for col in ["lambda_home", "lambda_away"]:
                if col in lambda_df.columns:
                    lambda_df[col] = lambda_df[col] * GLOBAL_LAMBDA_SCALE

        # Run QEPC simulation (single script)
        sim_results = run_qepc_simulation(lambda_df, num_trials=20000)

        return lambda_df, sim_results

    def on_game_change(change):
        if change["name"] != "value":
            return
        label = change["new"]
        if label is None:
            return

        with output_area:
            output_area.clear_output()

            row = upcoming_games[upcoming_games["Label"] == label].iloc[0]
            away = row["Away Team"]
            home = row["Home Team"]
            dt = row["gameDate"]

            print(f"🏀 QEPC Single Game View")
            print(f"{dt:%Y-%m-%d %I:%M %p} – {away} @ {home}")
            print("-" * 50)

            lambda_df, sim_results = run_qepc_for_game(row)

            # Expect sim_results to be a 1-row DataFrame
            sr = sim_results.iloc[0]

            home_prob = sr.get("Home_Win_Prob", None)
            away_prob = sr.get("Away_Win_Prob", None)
            exp_total = sr.get("Expected_Score_Total", None)
            exp_spread = sr.get("Expected_Spread", None)
            sim_home = sr.get("Sim_Home_Score", None)
            sim_away = sr.get("Sim_Away_Score", None)

            lam_home = lambda_df.iloc[0].get("lambda_home", None)
            lam_away = lambda_df.iloc[0].get("lambda_away", None)

            rows = []

            # Win probabilities
            if home_prob is not None and away_prob is not None:
                rows.append(
                    {
                        "Metric": "Win Prob",
                        "Home": f"{home_prob*100:.1f}%",
                        "Away": f"{away_prob*100:.1f}%",
                    }
                )

            # Expected score
            if sim_home is not None and sim_away is not None:
                rows.append(
                    {
                        "Metric": "Expected Score",
                        "Home": f"{sim_home:.1f}",
                        "Away": f"{sim_away:.1f}",
                    }
                )

            # Lambda μ
            if lam_home is not None and lam_away is not None:
                rows.append(
                    {
                        "Metric": "Lambda (μ)",
                        "Home": f"{lam_home:.1f}",
                        "Away": f"{lam_away:.1f}",
                    }
                )

            summary_df = pd.DataFrame(rows)

            print("Summary:")
            display(summary_df.style.hide_index())

            # Show lambda row with vol if present
            cols = [
                "Away Team",
                "Home Team",
                "lambda_away",
                "lambda_home",
                "vol_away",
                "vol_home",
            ]
            cols = [c for c in cols if c in lambda_df.columns]

            if cols:
                print("\nλ and volatility inputs:")
                display(lambda_df[cols])

            # Also print expected total/spread if available
            if exp_total is not None and exp_spread is not None:
                print(f"\nModel Total: {exp_total:.1f}")
                print(f"Model Spread (Home - Away): {exp_spread:.1f}")

    game_dropdown = widgets.Dropdown(
        options=list(upcoming_games["Label"]),
        description="Game:",
        layout=widgets.Layout(width="80%"),
    )
    game_dropdown.observe(on_game_change, names="value")

    display(game_dropdown, output_area)

    # Trigger initial display
    if len(game_dropdown.options) > 0:
        game_dropdown.value = game_dropdown.options[0]


-----